# Interactive Analysis of Major Canadian Banks

This notebook outlines the workflow and logic of the `major_banks_calc.py` script. It provides explanations and interactive code for analyzing the historical performance and risk of five major Canadian banks using Python, yfinance, pandas, numpy, plotly, and ipywidgets.

## 1. Import Required Libraries

Import the necessary libraries for data acquisition, manipulation, visualization, and interactivity. These include yfinance for stock data, pandas and numpy for analysis, plotly for plotting, and ipywidgets for interactive controls.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import webbrowser
from statsmodels.tsa.arima.model import ARIMA

## 2. Download Historical Stock Data

Use yfinance to download 5 years of daily closing prices for the five major Canadian banks: BMO, TD, RBC, CIBC, and Scotiabank. The data is retrieved as a pandas DataFrame for further analysis.

In [ ]:
# Download data for 5 major Canadian banks
tickers = ["BMO.TO", "TD.TO", "RY.TO", "CM.TO", "BNS.TO"]
bankData = yf.download(tickers, period="5y")['Close']
bankData.head()

## 3. Calculate Financial Metrics

Compute key financial metrics for each bank:
- **Daily Returns:** Percentage change in closing price from one day to the next.
- **Cumulative Returns:** Growth of $1 invested, calculated as the cumulative product of daily returns.
- **30-Day Rolling Volatility:** Standard deviation of daily returns over a 30-day window, representing risk.

In [ ]:
# Calculate daily returns, cumulative returns, and rolling volatility
dailyReturns = bankData.pct_change()
cumulativeReturns = (1 + dailyReturns).cumprod()
rollingVolatility = dailyReturns.rolling(window=30).std().dropna()

# Display sample metrics
display(dailyReturns.tail(3))
display(cumulativeReturns.tail(3))
display(rollingVolatility.tail(3))

## 4. Define Visualization Colors and Labels

Set up dictionaries to map each ticker to a specific color and a readable bank name. This ensures consistent and clear labeling in all plots.

In [ ]:
# Define colors and labels for each bank
colors = {
    "BMO.TO": "blue",
    "TD.TO": "green",
    "RY.TO": "gold",
    "CM.TO": "red",
    "BNS.TO": "pink"
}

labels = {
    "BMO.TO": "BMO",
    "TD.TO": "TD",
    "RY.TO": "RBC",
    "CM.TO": "CIBC",
    "BNS.TO": "Scotiabank"
}

## 5. Create Monte Carlo Forecast Function

Define a function to simulate future price paths using the Monte Carlo method. This function generates multiple possible future scenarios based on historical mean and standard deviation of returns, and returns the median and 5-95% confidence intervals for the forecasted prices.

In [ ]:
def monteCarloForecast(lastPrice, meanReturn, stdReturn, nDays=60, nSim=1000):
    """
    Simulate future price paths using Monte Carlo method.
    Returns future dates, median, lower (5%), and upper (95%) percentiles.
    """
    simPrices = np.zeros((nDays, nSim))
    for i in range(nSim):
        price = lastPrice
        for j in range(nDays):
            price = price * (1 + np.random.normal(meanReturn, stdReturn))
            simPrices[j, i] = price
    lower = np.percentile(simPrices, 5, axis=1)
    upper = np.percentile(simPrices, 95, axis=1)
    median = np.percentile(simPrices, 50, axis=1)
    futureDates = pd.date_range(start=bankData.index[-1] + pd.Timedelta(days=1), periods=nDays, freq='B')
    return futureDates, median, lower, upper

## 6. Visualize Metrics with Plotly

Create a function to plot the selected metric (cumulative returns, daily returns, or rolling volatility) for one or all banks using Plotly line charts. Optionally, overlay the Monte Carlo forecast results for a selected bank.

In [ ]:
def plot_bank_metric(selectedBank, selectedMetric, showForecast):
    """
    Plot the selected metric for the chosen bank(s).
    Optionally overlay Monte Carlo forecast for cumulative returns.
    """
    if selectedMetric == 'cumulativeReturns':
        dataToPlot = cumulativeReturns
        yLabel = "Cumulative Return ($1 Invested)"
    elif selectedMetric == 'dailyReturns':
        dataToPlot = dailyReturns
        yLabel = "Daily Return"
    elif selectedMetric == 'rollingVolatility':
        dataToPlot = rollingVolatility
        yLabel = "Volatility (Std Dev of Daily Returns)"
    else:
        raise ValueError("Invalid metric selected.")

    fig = go.Figure()

    banksToPlot = dataToPlot.columns if selectedBank == 'All' else [selectedBank]
    for bank in banksToPlot:
        fig.add_trace(go.Scatter(
            x=dataToPlot.index,
            y=dataToPlot[bank],
            mode='lines',
            name=labels[bank],
            line=dict(color=colors[bank], width=2)
        ))

    # Add Monte Carlo forecast if requested
    if showForecast and selectedBank != 'All' and selectedMetric == 'cumulativeReturns':
        lastPrice = cumulativeReturns[selectedBank].iloc[-1]
        meanReturn = dailyReturns[selectedBank].mean()
        stdReturn = dailyReturns[selectedBank].std()
        futureDates, median, lower, upper = monteCarloForecast(lastPrice, meanReturn, stdReturn)
        # Median forecast
        fig.add_trace(go.Scatter(
            x=futureDates,
            y=median,
            mode='lines',
            name=f"{labels[selectedBank]} Forecast Median",
            line=dict(color=colors[selectedBank], width=2, dash='dash')
        ))
        # Confidence interval
        fig.add_trace(go.Scatter(
            x=np.concatenate([futureDates, futureDates[::-1]]),
            y=np.concatenate([upper, lower[::-1]]),
            fill='toself',
            fillcolor='rgba(0,0,0,0.1)',
            line=dict(color='rgba(0,0,0,0)'),
            showlegend=True,
            name=f"{labels[selectedBank]} 5-95% Forecast"
        ))

    fig.update_layout(
        title=f"{selectedMetric.replace('daily', 'Daily').replace('cumulative', 'Cumulative').replace('Returns',' Returns').replace('Volatility',' Volatility')} of Selected Banks",
        xaxis_title="Date",
        yaxis_title=yLabel,
        hovermode="x unified",
        legend_title="Bank",
        template="plotly_white"
    )
    fig.show()

## 7. Interactive Controls with ipywidgets

Add dropdowns and checkboxes using ipywidgets to select the bank, metric, and toggle the Monte Carlo forecast. The plot updates interactively based on user selections.

In [ ]:
# Interactive widgets for bank, metric, and forecast selection
bank_options = [(labels[t], t) for t in tickers] + [("All Banks", "All")]
metric_options = [
    ("Cumulative Returns", "cumulativeReturns"),
    ("Daily Returns", "dailyReturns"),
    ("Rolling Volatility", "rollingVolatility")
]

bank_dropdown = widgets.Dropdown(
    options=bank_options,
    value="All",
    description="Bank:"
)

metric_dropdown = widgets.Dropdown(
    options=metric_options,
    value="cumulativeReturns",
    description="Metric:"
)

forecast_checkbox = widgets.Checkbox(
    value=False,
    description="Show Monte Carlo Forecast"
)

def update_plot(change=None):
    clear_output(wait=True)
    display(ui)
    plot_bank_metric(
        selectedBank=bank_dropdown.value,
        selectedMetric=metric_dropdown.value,
        showForecast=forecast_checkbox.value
    )

bank_dropdown.observe(update_plot, names='value')
metric_dropdown.observe(update_plot, names='value')
forecast_checkbox.observe(update_plot, names='value')

ui = widgets.VBox([bank_dropdown, metric_dropdown, forecast_checkbox])
display(ui)
update_plot()